In [3]:
# Convert whole datset into one csv file
import csv
import pandas as pd

result_attributes=['MatchID','Home_team','Away_team','Score_home','Score_away','Possession_home','Possession_away','Shots_on_target_home','Shots_on_target_away','Shots_home','Shots_away','Touches_home','Touches_away','Passes_home','Passes_away','Tackles_home','Tackles_away','Clearances_home','Clearances_away','Corners_home','Corners_away','Offsides_home','Offsides_away','Yellow_cards_home','Yellow_cards_away','Fouls_conceded_home','Fouls_conceded_away','Red_cards_home','Red_cards_away']

playerstats=['Clean sheets','Goals Conceded','Tackles','Tackle success %','Last man tackles','Blocked shots','Interceptions','Clearances','Headed Clearance','Clearances off line','Recoveries','Duels won','Duels lost','Successful 50/50s','Aerial battles won','Aerial battles lost','Own goals','Errors leading to goal','Assists','Passes','Passes per match','Big Chances Created','Crosses','Cross accuracy %','Through balls','Accurate long balls','Yellow cards','Red cards','Fouls','Offsides','Goals','Headed goals','Goals with right foot','Goals with left foot','Hit woodwork','Goals per match','Penalties scored','Freekicks scored','Shots','Shots on target','Shooting accuracy %','Big chances missed','Saves','Penalties Saved','Punches','High Claims','Catches','Sweeper clearances','Throw outs','Goal Kicks']


data_attributes=['MatchID','Score_home','Score_away','Clean sheets_home','Goals Conceded_home','Tackles_home','Tackle success %_home','Last man tackles_home','Blocked shots_home','Interceptions_home','Clearances_home','Headed Clearance_home','Clearances off line_home','Recoveries_home','Duels won_home','Duels lost_home','Successful 50/50s_home','Aerial battles won_home','Aerial battles lost_home','Own goals_home','Errors leading to goal_home','Assists_home','Passes_home','Passes per match_home','Big Chances Created_home','Crosses_home','Cross accuracy %_home','Through balls_home','Accurate long balls_home','Yellow cards_home','Red cards_home','Fouls_home','Offsides_home','Goals_home','Headed goals_home','Goals with right foot_home','Goals with left foot_home','Hit woodwork_home','Goals per match_home','Penalties scored_home','Freekicks scored_home','Shots_home','Shots on target_home','Shooting accuracy %_home','Big chances missed_home','Saves_home','Penalties Saved_home','Punches_home','High Claims_home','Catches_home','Sweeper clearances_home','Throw outs_home','Goal Kicks_home','Clean sheets_away','Goals Conceded_away','Tackles_away','Tackle success %_away','Last man tackles_away','Blocked shots_away','Interceptions_away','Clearances_away','Headed Clearance_away','Clearances off line_away','Recoveries_away','Duels won_away','Duels lost_away','Successful 50/50s_away','Aerial battles won_away','Aerial battles lost_away','Own goals_away','Errors leading to goal_away','Assists_away','Passes_away','Passes per match_away','Big Chances Created_away','Crosses_away','Cross accuracy %_away','Through balls_away','Accurate long balls_away','Yellow cards_away','Red cards_away','Fouls_away','Offsides_away','Goals_away','Headed goals_away','Goals with right foot_away','Goals with left foot_away','Hit woodwork_away','Goals per match_away','Penalties scored_away','Freekicks scored_away','Shots_away','Shots on target_away','Shooting accuracy %_away','Big chances missed_away','Saves_away','Penalties Saved_away','Punches_away','High Claims_away','Catches_away','Sweeper clearances_away','Throw outs_away','Goal Kicks_away']


def cts(x):
    if x.dtypes=='object':
        return x.str.replace(',','').astype(float)
    else:
        return x

dataset=[]
for year in range(2013,2023):
    result_csv_file=pd.read_csv('./Result Scrap/result-{year}.csv'.format(year=str(year)))
    stat_csv_file=pd.read_csv('./PlayerStat Scrap/Stats-{year}.csv'.format(year=str(year)))
    lineup_csv_file=open('./Listup Scrap/Listup-{year}.csv'.format(year=str(year)),'r')
    lineup_reader=csv.reader(lineup_csv_file)
    lineup_reader.__next__()
    final_data=pd.DataFrame(columns=data_attributes)

    for lineup in lineup_reader:
        results=result_csv_file.loc[result_csv_file['MatchID']==int(lineup[0])][['MatchID','Score_home','Score_away']]
        home_stat=[]
        away_stat=[]
        for i in range(11):
            home_stat.append(stat_csv_file.loc[stat_csv_file['Name'] == lineup[i+1]].drop(columns=['playerID','Name']))
        home_stats=pd.concat(home_stat)
        home_stats=home_stats.apply(cts)
        home_stat=pd.DataFrame(home_stats.mean())
        if len(home_stat[home_stat[0]==0])>6: # if more than 6 stats are 0, error occured, so skip
            continue
        home_stat=home_stat.transpose().rename(columns=lambda x:x+'_home')

        for i in range(11):
            away_stat.append(stat_csv_file.loc[stat_csv_file['Name'] == lineup[i+12]].drop(columns=['playerID','Name']))
        away_stats=pd.concat(away_stat)
        away_stats=away_stats.apply(cts)
        away_stat=pd.DataFrame(away_stats.mean())
        if len(away_stat[away_stat[0]==0])>6:
            continue
        away_stat=away_stat.transpose().rename(columns=lambda x:x+'_away')
        data=[results.reset_index(drop=True),home_stat.reset_index(drop=True),away_stat.reset_index(drop=True)]
        new_data=pd.concat(data,axis=1,ignore_index=True)
        dataset.append(new_data)
    lineup_csv_file.close()
final_data=pd.concat(dataset,axis=0)
final_data.columns=data_attributes
final_data.to_csv('./Dataset.csv')